# CNN 10 Folds Cross Validation

From our paper on "Explainable Prediction of Acute Myocardial Infarction using Machine Learning and Shapley Values"

In [3]:
# Import libraries
import pandas as pd
import numpy as np
import pylab as plt
from keras import optimizers, losses, activations, models, regularizers
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, Flatten, GlobalMaxPool1D, GlobalAveragePooling1D, \
    concatenate
from keras.utils import to_categorical
from keras.models import load_model, Sequential
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, confusion_matrix
from sklearn.model_selection import train_test_split
import time
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

Using TensorFlow backend.


In [6]:
def make_model(learning_rate=0.001):
    # number of categories in our problem
    opt = optimizers.Adam(learning_rate)
    model = Sequential()
    # shape of input
    model.add (Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid", input_shape=(11,1)))
    model.add (Dropout(rate=0.1))
    model.add (Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid"))
    model.add (Dropout(rate=0.1))
    model.add (Convolution1D(64, kernel_size=3, activation=activations.relu, padding="valid"))
    model.add (Dropout(rate=0.1))
    model.add (Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid"))
    model.add (GlobalMaxPool1D())
    model.add (Dropout(rate=0.1))
    model.add (Dense(64, activation=activations.relu, name="dense_1", kernel_regularizer=regularizers.l2(l=0.1)))
    model.add (Dense(16, activation=activations.relu, name="dense_2", kernel_regularizer=regularizers.l2(l=0.1)))
    model.add (Dense(2, activation=activations.softmax, name="dense_3_ecg_view"))
    model.compile(optimizer=opt, loss=losses.binary_crossentropy, metrics=['acc'])
    return model

In [ ]:
file_path = "baseline_cnn_ecgview_kfolds.h5"

# Checkpoint the model's weight based on the accuracy of the model
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

# Set early stopping based on accuracy. It stops after 10 consecutive epochs of no accuracy improvement.
early = EarlyStopping(monitor="val_acc", mode="max", patience=10, verbose=1)

# Reduce learning rate based on accuracy. It reduces the rate after 7 consecutive epochs of no accuracy improvement.
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=7, verbose=2)

callbacks_list = [checkpoint, early, redonplat]

In [ ]:
# Load array
train = loadtxt('train.csv', delimiter=',')
test = loadtxt('test.csv', delimiter=',')

# Split array
train_x = train[:,:11]
test_x = test[:,:11]
train_y = train[:,11]
test_y = test[:,11]

x_re = np.vstack((train_x, test_x))
y_re = np.vstack((train_y[:, None], test_y[:, None]))

In [11]:
# 10-fold cross validation on the test data 
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

X_re = np.reshape(x_re, (x_re.shape[0], x_re.shape[1], 1))

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
cvscores = []

for train, test in kfold.split(X_re, y_re):
    model = make_model(learning_rate = 0.001)
    # Fit the model
    model.fit(X_re[train],to_categorical(y_re[train]), epochs=1000, verbose=0, callbacks=callbacks_list, validation_split=0.1)
    # Evaluate the model
    scores = model.evaluate(X_re[test], to_categorical(y_re[test]), verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))


Epoch 00001: val_acc improved from -inf to 0.61715, saving model to baseline_cnn_ecgview_kfolds.h5

Epoch 00002: val_acc did not improve from 0.61715

Epoch 00003: val_acc improved from 0.61715 to 0.66832, saving model to baseline_cnn_ecgview_kfolds.h5

Epoch 00004: val_acc did not improve from 0.66832

Epoch 00005: val_acc did not improve from 0.66832

Epoch 00006: val_acc did not improve from 0.66832

Epoch 00007: val_acc improved from 0.66832 to 0.68849, saving model to baseline_cnn_ecgview_kfolds.h5

Epoch 00008: val_acc improved from 0.68849 to 0.73058, saving model to baseline_cnn_ecgview_kfolds.h5

Epoch 00009: val_acc did not improve from 0.73058

Epoch 00010: val_acc did not improve from 0.73058

Epoch 00011: val_acc improved from 0.73058 to 0.80721, saving model to baseline_cnn_ecgview_kfolds.h5

Epoch 00012: val_acc did not improve from 0.80721

Epoch 00013: val_acc did not improve from 0.80721

Epoch 00014: val_acc did not improve from 0.80721

Epoch 00015: val_acc did not


Epoch 00014: val_acc did not improve from 0.86745

Epoch 00015: val_acc did not improve from 0.86745

Epoch 00016: val_acc did not improve from 0.86745

Epoch 00016: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.

Epoch 00017: val_acc did not improve from 0.86745

Epoch 00018: val_acc did not improve from 0.86745

Epoch 00019: val_acc did not improve from 0.86745
Epoch 00019: early stopping
acc: 87.09%

Epoch 00001: val_acc did not improve from 0.86745

Epoch 00002: val_acc did not improve from 0.86745

Epoch 00003: val_acc did not improve from 0.86745

Epoch 00004: val_acc did not improve from 0.86745

Epoch 00005: val_acc did not improve from 0.86745

Epoch 00006: val_acc did not improve from 0.86745

Epoch 00007: val_acc did not improve from 0.86745

Epoch 00008: val_acc did not improve from 0.86745

Epoch 00009: val_acc did not improve from 0.86745

Epoch 00010: val_acc did not improve from 0.86745

Epoch 00011: val_acc did not improve from 0.86745

Epoch 0001


Epoch 00017: val_acc did not improve from 0.86745

Epoch 00018: val_acc did not improve from 0.86745

Epoch 00019: val_acc did not improve from 0.86745

Epoch 00020: val_acc did not improve from 0.86745

Epoch 00021: val_acc did not improve from 0.86745

Epoch 00022: val_acc did not improve from 0.86745

Epoch 00023: val_acc did not improve from 0.86745

Epoch 00024: val_acc did not improve from 0.86745

Epoch 00025: val_acc did not improve from 0.86745

Epoch 00026: val_acc did not improve from 0.86745

Epoch 00027: val_acc did not improve from 0.86745

Epoch 00028: val_acc did not improve from 0.86745

Epoch 00029: val_acc did not improve from 0.86745

Epoch 00030: val_acc did not improve from 0.86745

Epoch 00030: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.

Epoch 00031: val_acc did not improve from 0.86745

Epoch 00032: val_acc did not improve from 0.86745

Epoch 00033: val_acc did not improve from 0.86745
Epoch 00033: early stopping
acc: 86.61%

Epoch 0000